In [4]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils


In [6]:
landmarks_frame = pd.read_csv('faces/face_landmarks.csv')

In [10]:
#给出行数和列数
landmarks_frame.describe()


,part_0_x,part_0_y,part_1_x,part_1_y,part_2_x,part_2_y,part_3_x,part_3_y,part_4_x,part_4_y,...,part_63_x,part_63_y,part_64_x,part_64_y,part_65_x,part_65_y,part_66_x,part_66_y,part_67_x,part_67_y
count,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,...,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000
mean,108.289855,151.101449,108.913043,174.260870,111.521739,197.449275,116.739130,220.188406,126.130435,241.014493,...,203.347826,230.101449,223.144928,231.246377,203.782609,236.681159,194.768116,237.811594,185.884058,236.463768
std,69.881854,62.507210,69.022106,64.967787,68.261954,68.970483,67.423737,73.715924,66.070131,78.498875,...,66.086135,71.462072,68.569260,72.309467,66.235735,74.600208,65.354946,75.183750,64.607761,74.859983
min,-8.000000,48.000000,-6.000000,76.000000,-10.000000,86.000000,-8.000000,97.000000,6.000000,107.000000,...,72.000000,98.000000,85.000000,99.000000,72.000000,99.000000,69.000000,99.000000,66.000000,99.000000
25%,52.000000,109.000000,54.000000,123.000000,56.000000,141.000000,60.000000,165.000000,70.000000,183.000000,...,164.000000,184.000000,182.000000,187.000000,163.000000,187.000000,150.000000,188.000000,141.000000,187.000000
50%,100.000000,146.000000,102.000000,170.000000,106.000000,195.000000,113.000000,217.000000,124.000000,234.000000,...,199.000000,227.000000,221.000000,229.000000,198.000000,236.000000,188.000000,237.000000,181.000000,238.000000
75%,149.000000,183.000000,145.000000,205.000000,149.000000,231.000000,159.000000,255.000000,171.000000,283.000000,...,258.000000,269.000000,276.000000,269.000000,258.000000,274.000000,250.000000,277.000000,242.000000,274.000000
max,268.000000,392.000000,271.000000,427.000000,277.000000,464.000000,283.000000,501.000000,290.000000,536.000000,...,340.000000,507.000000,361.000000,511.000000,341.000000,516.000000,336.000000,518.000000,330.000000,516.000000


In [12]:
#查看前三个数据
landmarks_frame.head(3)

,image_name,part_0_x,part_0_y,part_1_x,part_1_y,part_2_x,part_2_y,part_3_x,part_3_y,part_4_x,...,part_63_x,part_63_y,part_64_x,part_64_y,part_65_x,part_65_y,part_66_x,part_66_y,part_67_x,part_67_y
0,0805personali01.jpg,27,83,27,98,29,113,33,127,39,...,93,136,100,141,93,135,89,135,84,134
1,1084239450_e76e00b7e7.jpg,70,236,71,257,75,278,82,299,90,...,148,311,179,308,149,312,137,314,128,312
2,10comm-decarlo.jpg,66,114,65,128,67,142,68,156,72,...,128,162,136,167,127,166,121,165,116,164


In [19]:
#第 8行数据
landmarks_frame.loc[3]

image_name    110276240_bec305da91.jpg
part_0_x                            42
part_0_y                           140
part_1_x                            45
part_1_y                           161
                        ...           
part_65_y                          204
part_66_x                          136
part_66_y                          208
part_67_x                          125
part_67_y                          209
Name: 3, Length: 137, dtype: object

In [15]:
#打印【4-6】的数据集 左闭右开 也就是 4、5
landmarks_frame.loc[range(4, 6)]

,image_name,part_0_x,part_0_y,part_1_x,part_1_y,part_2_x,part_2_y,part_3_x,part_3_y,part_4_x,...,part_63_x,part_63_y,part_64_x,part_64_y,part_65_x,part_65_y,part_66_x,part_66_y,part_67_x,part_67_y
4,1198_0_861.jpg,138,392,141,427,145,464,152,501,166,...,293,507,331,511,294,516,279,518,265,516
5,137341995_e7c48e9a75.jpg,-4,129,1,178,10,231,30,276,59,...,204,238,260,241,210,281,188,286,166,286
